In [ ]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout
from keras.models import Model
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications.mobilenet import MobileNet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
img_rows, img_cols = 150, 150

# Re-loads the MobileNet model without the top or FC layers
MobileNet = MobileNet(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# We freeze the layers of MobilNet
for layer in MobileNet.layers:
    layer.trainable = False
    
for (i,layer) in enumerate(MobileNet.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

17235968/17225924 [==============================] - 0s 0us/step
0 InputLayer False
1 Conv2D False
2 BatchNormalization False
3 ReLU False
4 DepthwiseConv2D False
5 BatchNormalization False
6 ReLU False
7 Conv2D False
8 BatchNormalization False
9 ReLU False
10 ZeroPadding2D False
11 DepthwiseConv2D False
12 BatchNormalization False
13 ReLU False
14 Conv2D False
15 BatchNormalization False
16 ReLU False
17 DepthwiseConv2D False
18 BatchNormalization False
19 ReLU False
20 Conv2D False
21 BatchNormalization False
22 ReLU False
23 ZeroPadding2D False
24 DepthwiseConv2D False
25 BatchNormalization False
26 ReLU False
27 Conv2D False
28 BatchNormalization False
29 ReLU False
30 DepthwiseConv2D False
31 BatchNormalization False
32 ReLU False
33 Conv2D False
34 BatchNormalization False
35 ReLU False
36 ZeroPadding2D False
37 DepthwiseConv2D False
38 BatchNormalization False
39 ReLU False
40 Conv2D False
41 BatchNormalization False
42 ReLU False
43 DepthwiseConv2D False
44 BatchNormalization F

In [ ]:
def lw(bottom_model, classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dense(classes,activation='softmax')(top_model)
    return top_model

In [ ]:
# Set our class number to 7 (Angry, Disgust, Fear, Happy, Neutral, Sad, Surprise)
classes = 7

FC_Head = lw(MobileNet, classes)

model = Model(inputs = MobileNet.input, outputs = FC_Head)


print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 75, 75, 32)        864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 75, 75, 32)       128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 75, 75, 32)        0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 75, 75, 32)       288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 75, 75, 32)       128       
 ation)                                                      

In [ ]:
path = "/content/drive/MyDrive/Mini Project 5th Semester/VisualData/"

datagen = ImageDataGenerator(
                    rescale=1./255,
                    validation_split = 0.2)

train_generator = datagen.flow_from_directory(
    path,
    target_size=(150,150),
    batch_size = 32,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    path,
    target_size=(150,150),
    batch_size = 32,
    class_mode='categorical'
)

Found 480 images belonging to 7 classes.
Found 480 images belonging to 7 classes.


In [ ]:
checkpoint = ModelCheckpoint("/content/drive/MyDrive/Mini Project 5th Semester/Speech_Emotion1.h5", # Putting CheckPoint to save model only when val_loss improves
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(learning_rate=0.01),
              metrics = ['accuracy'])

train_samples = 480
validation_samples = 480

epochs = 200
batch_size = 32

history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_samples // batch_size,
    epochs = epochs,
    callbacks = checkpoint,
    validation_data = validation_generator,
    validation_steps = validation_samples // batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/200
15/15 [==============================] - ETA: 0s - loss: 39.1088 - accuracy: 0.1875
Epoch 00001: val_loss improved from inf to 2.05505, saving model to /content/drive/MyDrive/Mini Project 5th Semester/Speech_Emotion1.h5
15/15 [==============================] - 71s 5s/step - loss: 39.1088 - accuracy: 0.1875 - val_loss: 2.0550 - val_accuracy: 0.2500
Epoch 2/200
15/15 [==============================] - ETA: 0s - loss: 1.9297 - accuracy: 0.2396
Epoch 00002: val_loss improved from 2.05505 to 1.88371, saving model to /content/drive/MyDrive/Mini Project 5th Semester/Speech_Emotion1.h5
15/15 [==============================] - 19s 1s/step - loss: 1.9297 - accuracy: 0.2396 - val_loss: 1.8837 - val_accuracy: 0.2500
Epoch 3/200
15/15 [==============================] - ETA: 0s - loss: 1.8336 - accuracy: 0.2521
Epoch 00003: val_loss improved from 1.88371 to 1.82998, saving model to /content/drive/MyDrive/Mini Project 5th Semester/Speech_Emotion1.h5
15/15 [==============================] 